<a href="https://colab.research.google.com/github/Rachana901070/ai/blob/main/ai_internal1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# depth limit search
def dls(graph,node,limit,goal,path=None):
  if path is None:
    path=[]
  path.append(node)
  if node==goal:
    return path
  if limit<=0:
    path.pop()
    return None
  for n in graph.get(node,[]):
    r=dls(graph,n,limit-1,goal,path)
    if r is not None:
      return r
  path.pop()
  return None
graph={'A':['B','C'],'B':['D','E'],'C':['F'],'D':[],'E':['F'],'F':[]}
print(dls(graph,'A',2,'F'))

['A', 'C', 'F']


In [ ]:
# uniform cost search
import heapq
def ucs(graph,start,goal):
  pq=[(0,[start])]
  visited=set()
  while pq:
    cost,path=heapq.heappop(pq)
    node=path[-1]
    if node==goal:
      return path,cost
    if node not in visited:
      visited.add(node)
      for n,sc in graph.get(node,[]):
        if n not in visited:
          np=list(path)
          np.append(n)
          heapq.heappush(pq,(cost+sc,np))
  return None,float("inf")
graph={'A':[['B',1],['C',2]],'B':[['D',3],['E',4]],'C':[['F',5]],'D':[],'E':[['F',6]],'F':[]}
print(ucs(graph,'A','F'))


(['A', 'C', 'F'], 7)


In [ ]:
#dfs
def dfs(graph,start,visited=None):
  if visited is None:
    visited=set()
  visited.add(start)
  print(start)
  for n in graph[start]-visited:
    dfs(graph,n,visited)
graph={'A':set(['B','C']),'B':set(['D','E']),'C':set(['F']),'D':set([]),'E':set(['F']),'F':set([])}
dfs(graph,'A')

A
B
D
E
F
C


In [ ]:
#bfs
v=[]
queue=[]
def bfs(graph,node,v):
  v.append(node)
  queue.append(node)
  while queue:
    m=queue.pop(0)
    print(m)
    for n in graph[m]:
      if n not in v:
        v.append(n)
        queue.append(n)
graph={'A':set(['B','C']),'B':set(['D','E']),'C':set(['F']),'D':set([]),'E':set(['F']),'F':set([])}
bfs(graph,'A',v)

A
B
C
D
E
F


In [ ]:
# water jug problem
def water_jug_problem(jug1,jug2,target):
  j1 = 0
  j2 = 0
  actions = [("fill",1),("fill,2"),("empty",1),("empty",2),("pour",1,2),("pour",2,1)]
  visited = set()
  queue = [(j2,j1,[])]
  while queue:
    j1, j2, seq = queue.pop(0)
    if (j1, j2) not in visited:
      visited.add((j1,j2))
      if j1 == target:
        return seq
      for action in actions:
        if action[0] == "fill":
          if action[1] == 1:
            next_state = (jug1,j2)
          else:
            next_state = (j1,jug2)
        elif action[0] == "empty":
          if action[1] == 1:
            next_state = (0,j2)
          else:
            next_state = (j1,0)
        else:
          if action[1] == 1:
            amount = min(j1,jug2-j2)
            next_state = (j1-amount,j2+amount)
          else:
            amount = min(j2,jug1-j1)
            next_state = (j1+amount,j2-amount)
        if next_state not in visited:
          next_seq=seq+[action]
          queue.append((next_state[0],next_state[1],next_seq))
  return None
result = water_jug_problem(4,3,2)
print(result)

[('fill', 1), ('pour', 1, 2), ('empty', 2), ('pour', 1, 2), ('fill', 1), ('pour', 1, 2)]


In [1]:
import heapq

# Goal state
goal_state = (1, 2, 3,
              4, 5, 6,
              7, 8, 0)   # 0 represents the blank tile

# Print puzzle in grid format
def print_board(state):
    for i in range(0, 9, 3):
        print(state[i:i+3])
    print()

# Check solvability using inversion count
def is_solvable(state):
    arr = [x for x in state if x != 0]
    inversions = 0
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                inversions += 1
    print(inversions)
    return inversions % 2 == 0

# Manhattan distance heuristic
def manhattan(state):
    distance = 0
    for i, value in enumerate(state):
        if value == 0:
            continue
        goal_index = goal_state.index(value)
        x1, y1 = divmod(i, 3)
        x2, y2 = divmod(goal_index, 3)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

# Generate neighbors
def get_neighbors(state):
    neighbors = []
    idx = state.index(0)
    x, y = divmod(idx, 3)
    moves = []
    if x > 0: moves.append((-1, 0, "Up"))
    if x < 2: moves.append((1, 0, "Down"))
    if y > 0: moves.append((0, -1, "Left"))
    if y < 2: moves.append((0, 1, "Right"))

    for dx, dy, action in moves:
        new_x, new_y = x + dx, y + dy
        new_idx = new_x * 3 + new_y
        new_state = list(state)
        new_state[idx], new_state[new_idx] = new_state[new_idx], new_state[idx]
        neighbors.append((tuple(new_state), action))
    return neighbors

# Reconstruct path
def reconstruct_path(parent, end):
    actions = []
    cur = end
    while parent[cur][0] is not None:
        cur, move = parent[cur]
        actions.append(move)
    return actions[::-1]

# A* search
def astar(start):
    if not is_solvable(start):
        print("This puzzle configuration is unsolvable.")
        return None

    frontier = []
    heapq.heappush(frontier, (manhattan(start), 0, start))  # (f, g, state)
    parent = {start: (None, None)}
    g_cost = {start: 0}

    while frontier:
        f, g, state = heapq.heappop(frontier)
        if state == goal_state:
            return reconstruct_path(parent,state)

        for neighbor, action in get_neighbors(state):
            new_g = g + 1
            if neighbor not in g_cost or new_g < g_cost[neighbor]:
                g_cost[neighbor] = new_g
                f_cost = new_g + manhattan(neighbor)
                heapq.heappush(frontier, (f_cost, new_g, neighbor))
                parent[neighbor] = (state, action)
    return None

# Example usage
if __name__ == "__main__":
    start_state = (1, 8, 2,
                   0, 4, 3,
                   7, 6, 5)
    # Example input

    print("Start State:")
    print_board(start_state)

    solution = astar(start_state)

    if solution:
        print("Solution found in", len(solution), "moves:")
        print(solution)
    else:
        print("No solution exists.")


Start State:
(1, 8, 2)
(0, 4, 3)
(7, 6, 5)

10
Solution found in 9 moves:
['Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Down']
